<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11:34 - 12:09

In [ ]:
%load_ext autoreload
%autoreload 2

## About Gradaul Typing

Properties:
 1. Optional
 2. Does not catch type erros at runtime
 3. Does not enhance performance

In [ ]:
def show_count(count, word):
  if count == 1:
    return f'1 {word}'
  count_str = str(count) if count else 'no'
  return f'{count_str} {word}s'

In [ ]:
show_count(99, 'bird')

'99 birds'

In [ ]:
show_count(1, 'bird')

'1 bird'

In [ ]:
show_count(0, 'bird')

'no birds'

In [ ]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 82.4 MB/s eta 0:00:00


In [ ]:
!mypy messages.py

Success: no issues found in 1 source file


In [ ]:
from pytest import mark

from messages import show_count

@mark.parametrize('qty, expected', [
    (1, '1 part'),
    (2, '2 parts'),
])
def test_show_count(qty, expected):
  got = show_count(qty, 'part')
  assert got == expected

def test_show_count_zero():
  got = show_count(0, 'part')
  assert got == 'no parts'

In [ ]:
!mypy --disallow-untyped-defs messages_test.py

messages.py:1: error: Function is missing a type annotation  [no-untyped-def]
messages_test.py:9: error: Function is missing a type annotation  [no-untyped-def]
messages_test.py:13: error: Function is missing a return type annotation  [no-untyped-def]
messages_test.py:13: note: Use "-> None" if function does not return a value
Found 3 errors in 2 files (checked 1 source file)


In [ ]:
!mypy messages_test.py

Success: no issues found in 1 source file


In [ ]:
from typing import Optional

def show_count(count: int, singular: str, plural: Optional[str] = None)

## Types Are Defined by Supported Operations

In [ ]:
from collections import abc
def double(x: abc.Sequence):
  return x * 2

A type checker will reject that code (`double(x)` wher `x` has type `str`, `tuple`, ..., `array`) because the type checker only cares about what is explicitly declared, and `abc.Sequence` has no `__mul__`.

### Duck Typing
 - The view adopted by SmallTalk as well as Python, Javascript, and Ruby. In practice, it doesn't matter what the declared type of the object is, only what operations it actually supports. If I can invoke `birdie.quack()`, then the `birdie` is a duck in this context. By definition, duck typing is only enforced at runtime.

### Nominal Typing
 - Adopted by C++, Java and C#, supported by annotated Python. Objects and vars have types but objects only exist at runtime and the type checker only cares about the source code where variables are annotated with type hints.

In [ ]:
!mypy birds.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
Found 1 error in 1 file (checked 1 source file)


In [ ]:
# daffy.py
from birds import *

daffy = Duck()
alert(daffy)
alert_duck(daffy)
alert_bird(daffy)

In [ ]:
!mypy daffy.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
Found 1 error in 1 file (checked 1 source file)


In [ ]:
!python daffy.py

Quack!
Quack!
Quack!


In [ ]:
# woody.py

from birds import *

woody = Bird()
alert(woody)
alert_duck(woody)
alert_bird(woody)

In [ ]:
!mypy woody.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
woody.py:7: error: Argument 1 to "alert_duck" has incompatible type "Bird"; expected "Duck"  [arg-type]
Found 2 errors in 2 files (checked 1 source file)


In [ ]:
!python woody.py

Traceback (most recent call last):
  File "/content/woody.py", line 6, in <module>
    alert(woody)
  File "/content/birds.py", line 9, in alert
    birdie.quack()
AttributeError: 'Bird' object has no attribute 'quack'


9:48 -

In [ ]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 72.0 MB/s eta 0:00:00


In [ ]:
!mypy double_object.py

Success: no issues found in 1 source file


* Traditional OOP
  - Liskov Property: If an object of type `T2` substitutes an object of type `T1` and the program still behaves correctly, then `T2` is subtype of `T1`.
* Gradual Type System
  - consistent-with relationship
  - which applies wherever subtype-of applies, with special provision for type `Any`

  - Rules
    1. Given `T1` and a subtype `T2`, then `T2` is consistent with `T1` (Liskov substitution)
    2. Every type is consistent with `Any`: you can pass objects of every type to an argument declared of type `Any`.
    3. `Any` is consistent-with every type: you can always pass an object of type `Any` where an argument of another type is expected.

## Optional and Union Types

The construct `Optional[str]` is actually a shortcut for `Union[str, None]`

In [ ]:
from typing import Optional

def show_count(count: int, signular: str, plural: Optional[str] = None) -> str:


## Generic Collection

In [ ]:
def tokenize(text: str) -> list[str]:
  return text.upper().split()

## Tuple Types

Three ways to annotate tuple types:
 1. Tuples as records
 2. Tuples as records with named fields
 3. Tuples as immutable sequences

1. Tuples as records
: use `tuple` built-in and declare the types of the fields within `[]`

In [ ]:
!pip install geolib

In [ ]:
from geolib import geohash as gh # type: ignore
PRECISION = 9

def geohash(lat_lon: tuple[float, float]) -> str:
  return gh.encode(*lat_lon, PRECISION)

In [ ]:
shanghai = 31.2304, 121.4737
geohash(shanghai)

'wtw3sjq6q'

2. Tuples as records with named fields

In [ ]:
from typing import NamedTuple

from geolib import geohash as gh # type: ignore

PRECISION = 9

# NamedTuple is a factory for `tuple` subclasses
# so `Coordinate` is consistent-with tuple[float, float]
class Coordinate(NamedTuple):
  lat: float
  lon: float

def geohash(lat_lon: Coordinate) -> str:
  return gh.encode(*lat_lon, PRECISION)

geohash(shanghai)

In [ ]:
!mypy coordinates_named.py

coordinates_named.py:15: error: Argument 1 to "geohash" has incompatible type "tuple[float, float]"; expected "Coordinate"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


In [ ]:
geohash(shanghai)

'wtw3sjq6q'

3. Tuples as immutable Sequences

To annotate tuples of unspecified length that are used as immutable lists, you must specify a single type, followed by a comma and ...

In [ ]:
# ex:
tuple[int, ...] # is a tuple with int items

In [ ]:
from collections.abc import Sequence

def columnize(
    sequence: Sequence[str], num_columns: int = 0
) -> list[tuple[str, ...]]:
  if num_columns == 0:
    num_columns = round(len(sequence) ** 0.5)
  num_rows, remainder = divmod(len(sequence), num_columns)
  num_rows += bool(remainder)
  return [tuple(sequence[i::num_rows]) for i in range(num_rows)]

animals = 'drake fawn heron ibex koala lynx tahr xerus yak zapus'.split()
table = columnize(animals)
table

[('drake', 'koala', 'yak'),
 ('fawn', 'lynx', 'zapus'),
 ('heron', 'tahr'),
 ('ibex', 'xerus')]

## Generic Mapping

Generic mapping types are annotated as `MappingType[KeyType, ValueType]`

In [ ]:
index = name_index(32, 65)

NameError: name 'name_index' is not defined

In [ ]:
import sys
import re
import unicodedata
from collections.abc import Iterator

RE_WORD = re.compile(r'\w+')
STOP_CODE = sys.maxunicode + 1

# tokenize is a generator function (will be dealt in chapter 17)
def tokenize(text: str) -> Iterator[str]:
  for match in RE_WORD.finditer(text):
    yield match.group().upper()

def name_index(start: int = 32, end: int = STOP_CODE) -> dict[str, set[str]]:
  index: dict[str, set[str]] = {}
  for char in (chr(i) for i in range(start, end)):
    # walrus operator / it assigns the result of the unicodedata.name() call to
    # name and the whole expression evaluates to that result
    if name := unicodedata.name(char, ''):
      for word in tokenize(name):
        index.setdefault(word, set()).add(char)
  return index

In [ ]:
if name := unicodedata.name('&', ''):
  print(name)

AMPERSAND


STOP_CODE

In [ ]:
index = name_index(32, 65)

In [ ]:
index['SIGN']

{'#', '$', '%', '+', '<', '=', '>'}

In [ ]:
index['DIGIT']

{'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

## Abstract Base Classes

Rule of thumb: Be liberal in what you accpet (function argument) and be conservative in what you send (return type)

In [ ]:
from collections.abc import Mapping

# (1)
# def name2hex(name: str, color_map: Mapping[str, int]) -> str:

# (2)
# def name2hex(name: str, color_map: dict[str, int]) -> str:
# cannot accept UserDict (because UserDict and dict are sibblings)

The `numbers` package defines the so-called *numeric tower*.

* `Number`
* `Complex`
* `Real`
* `Rational`
* `Integer`

Those ABCs work perfectly well for runtime type checking, but they are not supported for static type checking.

## Iterable

The `typing.List` documentation recommends `Sequence` and `Iterable` for function parameter type hints.

In [ ]:
# ex
# def fsum(__seq: Iterable[float]) -> float:

In [ ]:
# replacer.py

from collections.abc import Iterable

FromTo = tuple[str, str] # type alias

def zip_replace(text: str, changes: Iterable[FromTo]) -> str:
  for from_, to in changes:
    text = text.replace(from_, to)
  return text

In [ ]:
l33t = [('a', '4'), ('e', '3'), ('i', '1'), ('o', '0')]
text = 'mad skilled noob powned leet'
# from replacer import zip_replace
zip_replace(text, l33t)

'm4d sk1ll3d n00b p0wn3d l33t'

## abc.Iterable vs. abc.Sequence

It is fairly common in modern Python to offer functions that accept an `Iterable` input even if they must process it completely to return a result. That gives the caller the option of providing input data as a generator instead of a prebuilt sequence, potentially saving a lot of memory if the number of input items is large.

## Parametrized Generics and TypeVar

generic type written as `list[T]` where `T` is a type variable that will be bound to a specific type with each usage.

In [ ]:
# sample.py
from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

T = TypeVar('T') # why it is required
# Ans: make the [] work on classes like Sequence[T]
#      but the name of the T variable inside the bracket must be defined somewhere

def sample(population: Sequence[T], size: int) -> list[T]:
  if size < 1:
    raise ValueError('size must be >= 1')

  result = list(population)
  shuffle(result)
  return result[:size]

In [ ]:
# mode_float.py
from collections import Counter
from collections.abc import Iterable
from decimal import Decimal
from fractions import Fraction

NumberT = TypeVar('NumberT', float, Decimal, Fraction)
def mode(data: Iterable[NumberT]) -> NumberT:
  pairs = Counter(data).most_common(1)
  if len(pairs) == 0:
    raise ValueError('no mode for empy data')
  return pairs[0][0]

In [ ]:
mode([1, 1, 2, 3, 3, 3, 3, 3, 4])

3

In [ ]:
mode([1, (1, 2, 3), (1, 2, 3), 3])

(1, 2, 3)

### AnyStr predifned type variable

15:32 -

In [ ]:
AnyStr = TypeVar('AnyStr', bytes, str)

### Static Protocol

 In OOP, the concept of a protocol as an informal interface is as old as Smalltlak, and is an essential part of Python. However, in the context of type hints, a protocol is `typing.Protocol` subclass defining an interface and a type checker can verify.

 In Python, a protocol definition is written as as a `typing.Protocol` subclass. However, classes that implement a protocol don't need to inherit, register or declare any relationship with the class that defines the protocol.

Suppose you wanna create a function `top(it, n)` that returns a largest `n` elements of the iterable `it`:

In [ ]:
def top(series: Iterable[T], length: int) -> list[T]:
  ordered = sorted(series, reverse=True)
  return ordered[:length]

Q. How to constrain T? It cannot be `Any` or `object` because the series must work with `sorted`. The `sorted` built-in actually accepts `Iterable[Any]` but that's because the optional parameter `key` takes a function that computes an arbitrary sort key from each element. What happens if you give `sorted` a list of plain objects but don't provide a `key` argument?

In [ ]:
l = [object() for _ in range(4)]
l

In [ ]:
sorted(l)

TypeError: '<' not supported between instances of 'object' and 'object'

In [ ]:
class Spam:
  def __init__(self, n):
    self.n = n
  def __lt__(self, other):
    return self.n < other.n
  def __repr__(self):
    return f"Spam({self.n})"

In [ ]:
l = [Spam(n) for n in range(5, 0, -1)]

In [ ]:
l

[Spam(5), Spam(4), Spam(3), Spam(2), Spam(1)]

In [ ]:
sorted(l)

[Spam(1), Spam(2), Spam(3), Spam(4), Spam(5)]

We need a type parameter limiting to types that implement `__lt__`. But there is no suitable type in `typing` or `abc` to use, so we need to create it.

In [ ]:
# comparable.py
from typing import Protocol, Any

class SupportsLessThan(Protocol): # a protocol is a subclass of `typing.protocol`
  def __lt__(self, other: Any) -> bool: ... # The body of the protocol has one or more method def
  # with ... in their bodies

A type `T` is consistent-with a protocol `P` if `T` implements all the methods defined in `P`, with matching type signatures.

In [ ]:
from collections.abc import Iterable
from typing import TypeVar

LT = TypeVar('LT', bound=SupportsLessThan)

def top(series: Iterable[LT], length: int) -> list[LT]:
  ordered = sorted(series, reverse=True)
  return ordered[:length]

In [ ]:
top([4, 1, 5, 2, 6, 7, 3], 30)

[7, 6, 5, 4, 3, 2, 1]

In [ ]:
l = 'mango peer apple kiwi banana'.split()
top(l, 3)

['peer', 'mango', 'kiwi']

In [ ]:
# top_test.py
from collections.abc import Iterator
from typing import TYPE_CHECKING

import pytest

from top import top

def test_top_tuples() -> None:
  fruit = 'mango peer apple kiwi banana'.split()
  series: Iterator[tuple[int, str]] = (
      (len(s), s) for s in fruit
  )
  length = 3
  expected = [(6, 'banana'), (5, 'mango'), (5, 'apple')]
  result = top(series, length)
  if TYPE_CHECKING:
    reveal_type(series)
    reveal_type(expected)
    reveal_type(result)
  assert result == expected

def test_top_objects_error() -> None:
  series = [object() for _ in range(4)]
  if TYPE_CHECKING:
    reveal_type(series)
  with pytest.raises(TypeError) as excinfo:
    top(series, 3)
  assert "'<' not supported" in str(excinfo.value)

In [ ]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 73.0 MB/s eta 0:00:00


In [ ]:
!mypy top_test.py

top_test.py:18: note: Revealed type is "typing.Iterator[tuple[builtins.int, builtins.str]]"
top_test.py:19: note: Revealed type is "builtins.list[tuple[builtins.int, builtins.str]]"
top_test.py:20: note: Revealed type is "builtins.list[tuple[builtins.int, builtins.str]]"
top_test.py:26: note: Revealed type is "builtins.list[builtins.object]"
top_test.py:28: error: Value of type variable "LT" of "top" cannot be "object"  [type-var]
Found 1 error in 1 file (checked 1 source file)


A key advantage of a protocol type over ABCs is that a type doesn't need any special declaration to be consistent-with a protocol type.

### Callable

In [ ]:
input('hello')

helloda


'da'

The interaction of generic type parameters with a type hierarchy introduces a new typing concept: variance.

In [ ]:
# Illustrating variance
from collections.abc import Callable

def update(
    probe: Callable[[], float],
    display: Callable[[float], None]
) -> None:
  temperature = probe()
  display(temperature)

def probe_ok() -> int:
  return 42

def display_wrong(temperature: int) -> None:
  print(hex(temperature))

update(probe_ok, display_wrong)

0x2a


In [ ]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 49.4 MB/s eta 0:00:00


In [ ]:
!mypy variance.py

variance.py:17: error: Argument 2 to "update" has incompatible type "Callable[[int], None]"; expected "Callable[[float], None]"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


In [ ]:
def display_ok(temperature: complex) -> None:
  print(temperature)

Formally, we say that `Callable[[], int]` is subtype-of `Callable[[], float]` as `int` is subtype-of float. This means that `Callable` is covariant on the return type because the subtype-of relationship of the types `int` and `float` is in the same direction.

In [ ]:
scores: list[float] = [2.0, 3.0]
a: list[int] = [2, 3]
scores = a

In [ ]:
!mypy variance_test.py

variance_test.py:3: error: Incompatible types in assignment (expression has type "list[int]", variable has type "list[float]")  [assignment]
variance_test.py:3: note: "List" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
variance_test.py:3: note: Consider using "Sequence" instead, which is covariant
Found 1 error in 1 file (checked 1 source file)


### NoReturn

This is a special type used only to annotate the return type of functions that never return. Usually, they exist to raise exceptions.

In [ ]:
def exit(__status: object = ...) -> NoReturn: ...

# Annotating Positional Only and Variadic Parameters

PEP 484 convention: prefix each positional-only parameter